#### Getting Predicted Data

In [1]:
import pandas as pd
import numpy as np

predicted_data = pd.read_parquet('../model\sample_predictions.parquet')
predicted_data

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\joshu\AppData\Local\Temp\ipykernel_14788\508343493.py:4: SyntaxWarning: invalid escape sequence '\s'
  predicted_data = pd.read_parquet('../model\sample_predictions.parquet')


,median
E01000001,0.0
E01000002,0.0
E01000003,0.0
E01000005,0.0
E01000006,0.0
...,...
E01035688,0.0
E01035689,0.0
E01035690,1.0
E01035691,1.0


#### Getting Observed Data

In [2]:
# Load the CSV files
metropolitan_df = pd.read_csv('observed_data/2025-03-metropolitan-street.csv')
city_of_london_df = pd.read_csv('observed_data/2025-03-city-of-london-street.csv')

# Combine both dataframes
combined_df = pd.concat([metropolitan_df, city_of_london_df], ignore_index=True)

# Filter to only include burglaries
burglary_df = combined_df[combined_df['Crime type'].str.lower() == 'burglary']

# Count the number of burglaries per LSOA
burglary_counts = burglary_df['LSOA code'].value_counts().reset_index()
burglary_counts.columns = ['LSOA code', 'count']
burglary_counts.set_index('LSOA code', inplace=True)
burglary_counts = burglary_counts.sort_index(ascending=True)

observed_data = burglary_counts
observed_data = observed_data.reset_index().set_index('LSOA code')[['count']]

observed_data

,count
LSOA code,
E01000003,1
E01000005,1
E01000007,4
E01000009,1
E01000013,2
...,...
E01035716,14
E01035717,6
E01035718,3


#### Function for Evaluation Metric

######

##### RMSE 

In [10]:
list_of_lsoa = predicted_data.index.tolist()

def compute_rmse(predictions, lsoa_codes, observed_df):
    """
    Compute CRPS between predicted samples and observed burglary counts.

    Parameters:
    - predictions: np.ndarray of shape (n_samples, n_LSOAs)
    - lsoa_codes: list of LSOA codes in the same order as columns of `predictions`
    - observed_df: DataFrame or Series indexed by LSOA with counts

    Returns:
    - average CRPS score over all LSOAs
    """
    def rmse_single(pred_samples, obs):
        pred_samples = np.asarray(pred_samples)
        term1 = np.mean((pred_samples - obs)**2)
        term2 = 0.5 * np.mean(np.abs(pred_samples[:, None] - pred_samples[None, :]))
        return term1 - term2

    all_lsoas = pd.Series(0, index=lsoa_codes)

    # Accept either a DataFrame with 'count' or a Series
    if isinstance(observed_df, pd.DataFrame):
        observed_series = observed_df['count']
    else:
        observed_series = observed_df

    observed_aligned = all_lsoas.copy()
    observed_aligned.update(observed_series)

    rmse_scores = []
    for i, lsoa in enumerate(lsoa_codes):
        obs = observed_aligned[lsoa]
        rmse = rmse_single(predictions[:, i], obs)
        rmse_scores.append(rmse)

    return np.mean(rmse_scores)



In [12]:
# list_of_lsoa must match predicted_data.columns exactly
list_of_lsoa = predicted_data.columns.tolist()

# Compute CRPS
rmse_score = compute_rmse(predicted_data.to_numpy(), list_of_lsoa, observed_data)
print(f"rmse Score: {rmse_score:.4f}")


rmse Score: 0.5356


average predictive distribution is, on average, 0.54 units away from the true crime count (in a probabilistic sense).

- predicts crime 0.54+- away from true prediction

##### CRPS